In [ ]:
import json
import pandas as pd
import numpy as np
import csv
import re

Importing json movie data 


In [ ]:
file_dir = r'C:\Users\prans\Desktop\U of T\Classes\unit 8 etl'

In [ ]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
# First 5 records
wiki_movies_raw[:5]
# Last 5 records
wiki_movies_raw[-5:]
# Some records in the middle
wiki_movies_raw[3600:3605]

Extract kaggle data


In [ ]:
kaggle_metadata = pd.read_csv("C:/Users/prans/Desktop/U of T/Classes/unit 8 etl/movies_metadata.csv")
ratings = pd.read_csv("C:/Users/prans/Desktop/U of T/Classes/unit 8 etl/ratings.csv")

In [ ]:
kaggle_metadata.head()

In [ ]:
ratings.head()

In [ ]:
ratings.sample(n=5)

Transforming Data and cleaning data


In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
wiki_movies_df.columns.tolist()

In [ ]:
#creating list comprehention with filter expression for movie directors and imdb link
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                    and 'No. of episodes' not in movie]
len(wiki_movies)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies)
wiki_movies_df.head()

Changing multiple language and merging the same column names


In [ ]:
#looking for arabic language for example
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

In [ ]:
#getting url of arabic language
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
#creating sorted list to see the columns 
sorted(wiki_movies_df.columns.tolist())

In [ ]:
# we see that thare are many alternative languages and is bad data 
# create an alternate titles dict and store it in an one column in movies

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')

    return movie


In [ ]:
#list comprehention to create clean movie 
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [ ]:
#creating wiki movies dataframe from clean_movies
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

Removing Duplicate rows

In [ ]:
#we will use reg expression
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))

In [ ]:
#remove null columns
#count of null values of each column using list comprehension
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

In [ ]:
#many columns have null value we need to make a l;ist of coluns having less than 90% null values and trim down our dataset
# we will use list comprehension
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
wiki_movies_df.head()

Before storing data in SQl we need to parse data in their data type 

In [ ]:
# looking at the data types of columns
wiki_movies_df.dtypes

In [ ]:
# changing dtype of boxoffice column
#making data series that drops missing values
box_office = wiki_movies_df['Box office'].dropna()
len(box_office)

In [ ]:
# regular expression only works on the string so lets check this by making a function
def is_not_a_string(x):
    box_office[box_office.map(is_not_a_string)]
    return type(x) != str
     


In [ ]:
# USING LAMBDA FUNCTIONS
box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
#converting list to a string and joinig them using lamda function
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#parse the box office data form 1 "$123.4 million/billion."
form_one = r'\$\d+\.?\d*\s*[mb]illion'
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()


In [ ]:
#parse the box office data form 2 "$123,456,789." 
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()


In [ ]:
# which entries do not fit form 1 or 2 ,creating a boolean series
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [ ]:
# this will throw an error!Pandas has element-wise logical operators:
#The element-wise logical "and" is the ampersand: &
#The element-wise logical "or" is the pipe: |

box_office[~matches_form_one & ~matches_form_two]

In [ ]:
#Fix Pattern Matches
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [ ]:
box_office.str.extract(f'({form_one}|{form_two})')

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
#Extract the values from box_office with str.extract and apply parse_dollars to the first column in the df returned by str.extract
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df['box_office']

In [ ]:
#Drop the box_office column
wiki_movies_df.drop('Box office', axis = 1, inplace=True)
wiki_movies_df.head()

In [ ]:
#Parse Budget Data
#Create a budget variable with the following code:

budget = wiki_movies_df['Budget'].dropna()

In [ ]:
#Convert any lists to strings:
udget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#hen remove any values between a dollar sign and a hyphen (for budgets given in ranges):
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
#Apply the same pattern matches from box_office to the budget_data without modifications
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False )
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
budget[~matches_form_one & ~matches_form_two]

In [ ]:
#Remove the citation references
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

In [ ]:
#Parse the budget values
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
#Drop the original Budget column
wiki_movies_df.drop('Budget', axis=1, inplace=True)
wiki_movies_df.head()

Parse Release Date

In [ ]:
#Make a variable that holds the non-null values of Release date in the data frame, converting lists to strings
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#The forms that will be parsed include:
#1.) full month name, one- to two-digit day, four-digit year (i.e. January 1, 2000)
#2.) four-digit year, two-digit month, two-digit day, with any separator (i.e. 2000-01-01)
#3.) full month name, four-digit year (i.e. January 2000)
#4.) four-digit year

date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[123]\d'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [ ]:
#Extract the dates
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)


Parse Running Time

In [ ]:
#Make a variable that holds the non-null values of Release date in the data frame, converting lists into strings
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
#Determine the number of running times that follow the format "100 minutes" using string boundaries
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

In [ ]:
#View what the other data formats look like
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
#Let's make this more general by only marking the beginning of the string, and accepting other abbreviations of "minutes" by only searching up to the letter "m."
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()

In [ ]:
#View the remaining running times
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
#Match all hour+minute patterns with one regular expression pattern:
#1.) start with one or more digits
#2.) have an optional space after the digit and before the letter 'h'
#3.) captuer all possible abbreviations of 'hour(s).' (We would make every letter in 'hours' optional except the 'h')
#4.) have an optional space after the 'hours' marker
#5.) have an optional number of digits for minutes
#Add capture groups and alternating characters to ensure that we extract the digits and allow for both possible patterns
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
#We need to convert this new data frame from strings to numeric values. Since we may have captured empty strings,
#we'll use the to_numeric() method and set the errors argument to 'coerce'. Coercing the errors will turn the empty strings
#into Not a Number (NaN). Then use fillna() to change all NaNs to zeros
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)



In [ ]:
#Apply a function that converts the hour capture groups and minute capture groups to minutes 
#if the pure minute capture group is zero and save the output to wiki_movies_df
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)



Clean the Kaggle Data

In [ ]:
kaggle_metadata.dtypes

In [ ]:
kaggle_metadata.columns.tolist()

Remove Bad Data

In [ ]:
kaggle_metadata['adult'].value_counts()

In [ ]:

kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

In [ ]:
#The best option is to keep the rows where 'adult' is 'False' and then drop the 'adult' column
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis='columns')
kaggle_metadata

Convert Data Types

In [ ]:
#Check the values of the 'video' column
kaggle_metadata['video'].value_counts()

In [ ]:
#Create a Boolean column and assign it back to 'video'
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
#Use the to_numeric() method to convert into numeric columns
#Set the errors argument to 'raise' so we'll know if there are any data that cannot be converted
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
#Convert release_data to datetime
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

Reasonability Checks on Rating Data

In [ ]:
#View the ratings data with the info() method
ratings.info(null_counts=True)

In [ ]:
#Specify that the origin is 'unix' and the time unit is in seconds
pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
#Assign the outputs to the timestamp column
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
#Check the statistics of the actual ratings to see if there are any major errors
#The quickest way to do so is to look at a histogram of the rating distributions
#Then use the describe() method to print out stats on central tendency and spread
pd.options.display.float_format = '{:20,.2f}'.format
ratings['rating'].plot(kind='hist')
ratings['rating'].describe()

In [ ]:
#Print out a list of columns so that we can identify which ones are redundant
#Use the suffix parameter to make it easier to identify which table each column came frome
movies_df = pd.merge(wiki_movies_df,  kaggle_metadata, on='imdb_id', suffixes=['_wiki', '_kaggle'])
movies_df.head()

In [ ]:
# Competing data:
# Wiki                     Movielens                Resolution
#-----------------------------------------------------------------------------------------------------
# title_wiki               title_kaggle             Drop Wikipedia
# running_time             runtime                  Keep Kaggle; fill in zeros with Wikipedia data
# budget_wiki              budget_kaggle            Keep Kaggle; fill in zeros with Wikipedia data
# box_office               revenue                  Keep Kaggle; fill in zeros with Wikipedia data
# release_date_wiki        release_date_kaggle      Drop Wikipedia
# Language                 original_language        Drop Wikipedia
# Production company(s)    production_companies     Drop Wikipedia

Title

In [ ]:
#View the titles
movies_df[['title_wiki', 'title_kaggle']]

In [ ]:
#Look at the rows in which the titles do not match
movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]

In [ ]:
#Both options look pretty good, but the Kaggle data looks just a little bit more consistent. Let's confirm there aren't any missing titles in the Kaggle data with the following code:
movies_df[(movies_df['title_kaggle'] == '') | (movies_df['title_kaggle'].isnull())]

##Since no titles are missing, we can drop the Wikipedia title. For now, we will only note the resolution in the chart

Runtime

In [ ]:
#The following code will fill in missing values with zero and make the scatter plot:
movies_df.fillna(0).plot(x='running_time', y='runtime', kind='scatter')

Budget

In [ ]:
#Since budget_wiki and budget_kaggle are numeric, we can make another scatter plot to compare values
movies_df.fillna(0).plot(x='budget_wiki', y='budget_kaggle', kind='scatter')


Box Office

In [ ]:
#box_office and revenue are also numeric, so we can proceed with another scatter plot
movies_df.fillna(0).plot(x='box_office', y='revenue', kind='scatter')

In [ ]:
#That looks pretty close, but we might be getting thrown off by the scale of that large data point. Let's look at the scatter plot for everything less than $1 billion in box_office.
movies_df.fillna(0)[movies_df['box_office'] < 10**9].plot(x='box_office', y='revenue', kind='scatter')

Release Date

In [ ]:
#Because release_date_wiki and release_date_kaggle aren't numeric data, we can't use a scatter plot.
#However, we can work around it with a regular line plot and change the style to put only dots
movies_df[['release_date_wiki', 'release_date_kaggle']].plot(x='release_date_wiki', y='release_date_kaggle', style='.')

In [ ]:
#Investigate the wild outlier around 2006. Choose some rough cutoff dates to single out the movie.
#Our paramters will include movies whose release date according to Wikipedia is after 1996, but whose release date according to Kaggle is before 1965
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')]

#Based on the output, it looks like "The Holiday" in the Wikipedia data got merged with "From Here to Eternity."

In [ ]:
#Retrieve the index of the row
movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index

In [ ]:
#Then we can drop that row like this:
movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & (movies_df['release_date_kaggle'] < '1965-01-01')].index)

In [ ]:
#Check if there are any null values
movies_df[movies_df['release_date_wiki'].isnull()]

#The Wikipedia data is missing release dates but the Kaggle data isn't so we will just drop the Wikipedia data

Language

In [ ]:
#Some values are stored as lists, we need to convert them to tuples so that the value_counts() method will work
movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts()

In [ ]:
#For the Kaggle data, there are no lists, so we can just run value_counts()
movies_df['original_language'].value_counts(dropna=False)

#Since the Kaggle data is in a relatively consistent and usable format, we'll use the Kaggle data as it saves time

Production Companies

In [ ]:
#Check a portion of the data. We can see that the Kaggle data is much more consiststent
movies_df[['Production company(s)', 'production_companies']]

Clean Up and Merge

In [ ]:
#First drop title_wiki, release_date_wiki, Language, [and Production company(s)]
movies_df.drop(columns=['title_wiki', 'release_date_wiki', 'Language'], inplace=True)

In [ ]:

#Make a function that fills in missing data for a column pair and then drops the redundant column
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column,inplace=True)

In [ ]:
movies_df.columns.to_list()

In [ ]:
#Run the function for the three column pairs that we decided to fill in zeros
fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
movies_df

In [ ]:
#After the merge, check to see that there aren't any columns with only one value. 
#Don't forget to convert lists to tuples for value_counts() to work
for col in movies_df.columns:
    lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
    value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
    num_values = len(value_counts)
    if num_values == 1:
        print(col)

In [ ]:
#Running the previous code, we see that 'video' only has one value. Show the output of 'video'
movies_df['video'].value_counts(dropna=False)

#Since it is false for every row, we don't need to include this column

In [ ]:
movies_df.columns.tolist()

In [300]:
#Lastly, reorder the columns to make the data set easier to read. One way to order them would be to consider the columns roughly in groups.
# 1.) Identifying information (IDs, titles, URLs, etc.)
# 2.) Quantitative facts (runtime, budget, revenue, etc.)
# 3.) Qualitative facts (genres, languages, country, etc.)
# 4.)Business data (production companies, distributors, etc.)
# 5.) (producers, director, cast, writers, etc.)
movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages','Country',
                       'production_companies','production_countries','Distributor',
                       'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                      ]]

In [301]:
#Finally, we need to rename the columns to be consistent.
movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)

In [302]:
movies_df.head()


,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,production_countries,distributor,producers,director,starring,cinematography,editors,writers,composers,based_on
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.00,"49,000,000.00",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",20th Century Fox,"[Steve Perry, Joel Silver]",Renny Harlin,"[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,Michael Tronick,"[David Arnott, James Cappe]","[Cliff Eidelman, Yello]","[Characters, by Rex Weiner]"
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.00,"6,000,000.00",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Avenue Pictures,"[Ric Kidney, Robert Redlin]",James Foley,"[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,Howard E. Smith,"[James Foley, Robert Redlin]",Maurice Jarre,"[the novel, After Dark, My Sweet, by, Jim Thom..."
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.00,"35,000,000.00",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",TriStar Pictures,Daniel Melnick,Roger Spottiswoode,"[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[John Bloom, Lois Freeman-Fox]","[John Eskow, Richard Rush]",Charles Gross,"[Air America, by, Christopher Robbins]"
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.00,"12,000,000.00",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Orion Pictures,Robert Greenhut,Woody Allen,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,Susan E. Morse,Woody Allen,NaN,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.00,"25,000,000.00",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Paramount Pictures,John Cornell,John Cornell,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,David Stiven,Paul Hogan,Maurice Jarre,NaN
